# Amazon SageMaker Managed Spot Training ワークフローを構築

このノートブックは、以下のようにインスタンスタイプのリストを実行時のパラメタとして指定すると、SageMaker 学習ジョブが正常に完了するまで指定されたインスタンスで学習ジョブを順に起動するワークフローを作成します。これにより、タイムアウトで学習ジョブが停止した場合でも、自動的に別のインスタンスタイプで学習ジョブを開始することができます。

```json
"InstanceList": [
    [
      "ml.p3.2xlarge",  # 使用したいインスタンス情報
      "spot"              # スポットインスタンス
    ],
    [
      "ml.m5.xlarge",
      "spot"
    ],
    [
      "ml.c5.xlarge",
      "ondemand"      # オンデマンドインスタンス
    ]
]
```


1. [背景](#背景)
1. [セットアップ](#セットアップ)
1. [S3 バケットの準備](#S3-バケットの準備)
1. [データの準備](#データの準備)
1. [学習ジョブの準備](#学習ジョブの準備)
1. [Step Functions ループ制御用 Lambda 関数の準備](#Step-Functions-ループ制御用-Lambda-関数の準備)
1. [AWS Step Functions の準備](#AWS-Step-Functions-の準備)
1. [AWS Step Functions Workflow の実行](#AWS-Step-Functions-Workflow-の実行)
1. [リソースの削除](#リソースの削除)


---

## 背景

Amazon SageMaker Managed Spot Training は、スポットインスタンスを使ってコスト効率よく機械学習モデルを学習するための機能です。スポットインスタンスは、通常のオンデマンドインスタンスよりも安く利用できますが、空きがなくなるとジョブが中断したり、スポットインスタンスを確保できずジョブが開始しない可能性があります。特に人気の高い GPU インスタンスは、スポットインスタンスの空きがないことも多くあります。そこで、Managed Spot Training ワークフローを作成して、学習ジョブ実行時に複数のインスタンスタイプを指定しておくことで、スポットインスタンスの空きがなく学習ジョブが正常完了しない場合は順次指定したインスタンスで学習ジョブを実行していき、ワークフロー完了時にいずれかの学習ジョブが正常完了するようにします。

本ノートブックは、以下のようなワークフローを AWS Step Functions を使って構築します。ワークフロー実行時に、インスタンスタイプと、スポットインスタンスかオンデマンドインスタンスを示す文字列のリストが渡されるため、その値を確認してスポット学習か通常の学習ジョブのいずれかを指定されたインスタンスタイプで開始します。学習ジョブが終了したら終了ステータスを確認し、正常終了であればワークフローを終了します。何らかのエラーが発生していれば、次に指定されたインスタンスタイプと学習ジョブのタイプで学習ジョブを開始します。この処理を、リスト内の項目の数だけ繰り返します。

<img src="workflow.png" width="80%">

---


## セットアップ
### Step Functions Data Science SDK をインストール

以下のセルを実行したら、**メニューの「Kernel」->「Restart」をクリックしてカーネルを再起動してください。**再起動後は以下のセルを再度実行する必要はないので、その下から作業を再開してください。

In [ ]:
%%sh
pip install -U awscli boto3 "sagemaker>=2.0.0"
pip install -U "stepfunctions==2.3.0"


SageMaker セッションを作成し、設定を開始します。

In [ ]:
import boto3
from datetime import datetime
from dateutil import tz
import json
import os
import pandas as pd
import sagemaker
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput
from time import sleep
import utility

project_name = 'sagemaker-spot'
user_name = 'demo'

JST = tz.gettz('Asia/Tokyo')
timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
iam_client = boto3.client('iam', region_name=region)
sfn_client = boto3.client('stepfunctions', region_name=region)

role = sagemaker.get_execution_role()
account_id = boto3.client('sts').get_caller_identity().get('Account')

sagemaker_policy_name = project_name + '-' + user_name + '-policy'
prefix = f'sagemaker/{project_name}/{user_name}'
bucket_name = project_name + '-' + user_name + '-' + timestamp

s3_client = boto3.client('s3', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)

policy_arn_list = []
role_name_list = []
lambda_function_list = []

role_name = role.split('/')[-1]
iam_console_url = f'https://{region}.console.aws.amazon.com/iamv2/home#/roles/details/{role_name}?section=permissions'

from IPython.display import display, Markdown
text = f"""
以下の手順で IAM 関連の設定を実施してください。
1. <a href=\"policy/sagemaker-policy.json\" target=\"_blank\">policy/sagemaker-policy.json</a> の中身をコピー
1. <a href=\"https://{region}.console.aws.amazon.com/iam/home#/policies$new?step=edit\" target=\"_blank\">IAM Policy の作成</a>をクリックし、**JSON** タブをクリックしてから手順1でコピーした JSON をペーストして右下の **次のステップ：タグ** ボタンをクリック
1. 右下の **次のステップ：確認** ボタンをクリック
1. **名前** に **「{sagemaker_policy_name}」** を記載して、右下の **ポリシーの作成** ボタンをクリック
1.  <a href=\"{iam_console_url}\" target=\"_blank\">ノートブックインスタンスにアタッチされた IAM Role</a> を開く
1. **許可を追加** ボタンをクリックして **ポリシーをアタッチ** を選択
1. **その他の許可ポリシー** の検索ボックスで手順4 で作成した {sagemaker_policy_name} を検索して横にあるチェックボックスをオンにする
1. **ポリシーのアタッチ** をクリック
"""
display(Markdown(text))

## S3 バケットの準備
### Job 生成物格納用 S3 バケットの準備

SageMaker Jobs が生成したデータやモデルなどを保存する S3 バケットを作成します。セキュリティのため暗号化を有効にします。

In [ ]:
utility.create_bucket(bucket_name, region, account_id)

## データの準備

このサンプルノートブックでは、手書き数字のデータセット MNIST を使用します。

### データの取得

AWS が用意した S3 バケットからデータをダウンロードして展開します。

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
!tar -xvzf  mnist_png.tgz

ダウンロードしたデータを pt 形式で保存します。このノートブックでは pt 形式を使用しますが、データ形式はご自身の使用する学習スクリプトに合わせて変更してください。

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

data_dir =  'data'
training_dir = 'mnist_png/training'
test_dir = 'mnist_png/testing'

os.makedirs(data_dir, exist_ok=True)

training_data = datasets.ImageFolder(root=training_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

training_data_loader = DataLoader(training_data, batch_size=len(training_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

### データを S3 にアップロードする
SageMaker 学習ジョブを使う場合は、学習データが S3 に保存されている必要があります。データセットを S3 にアップロードするには、 `sagemaker.Session.upload_data` 関数を使用します。 戻り値として入力した S3 のロケーションは、後で学習ジョブを実行するときに使用します。

In [ ]:
inputs = sagemaker_session.upload_data(path=data_dir, bucket=bucket_name, key_prefix=os.path.join(prefix, 'data'))
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## 学習ジョブの準備
### 学習スクリプトを S3 にアップロードする

学習ジョブで使用する学習スクリプトを tar.gz で圧縮して S3 にアップロードします。SageMaker Python SDK の Estimator を使う場合はこの処理に相当する部分は SDK がやってくれますが、Step Functions から学習ジョブを実行する際は低レイヤの API が使用されるため、自分でスクリプトを S3 にアップロードしておく必要があります。

スポット学習ジョブが中断された後、再度スポットインスタンスに空きがでて、指定された待機時間内であればジョブが再開します。その際に、チェックポイントを利用することで、モデルの学習を中断前の続きから実施することができます。チェックポイント機能を活用するには、asl.json の `CheckpointConfig` にチェックポイントを保存するためのローカルパスと S3 パスを指定し、学習スクリプトにチェックポイントの保存と読み込みのコードを書けば OK です。このサンプルではチェックポイントを利用するよう構成されています。スポット学習の状態遷移については [こちらのドキュメント](https://docs.aws.amazon.com/ja_jp/sagemaker/latest/dg/model-managed-spot-training.html#model-managed-spot-training-status) を参照してください。


In [ ]:
timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M')

TRAINNING_SCRIPT_LOCATION = "source.tar.gz"
!cd code/sagemaker && tar zcvf ../../$TRAINNING_SCRIPT_LOCATION train.py

train_code = sagemaker_session.upload_data(
    TRAINNING_SCRIPT_LOCATION,
    bucket=bucket_name,
    key_prefix=os.path.join(project_name, user_name, "train/code", timestamp),
)
train_code

### 学習時に使用するコンテナイメージの URI を取得する

In [ ]:
from sagemaker.image_uris import retrieve

pytorch_training_image_uri = retrieve('pytorch',
                                       region,
                                       version='1.10',
                                       py_version='py38',
                                       instance_type = 'ml.m5.xlarge',
                                       accelerator_type=None,
                                       image_scope='training')
pytorch_training_image_uri

## Step Functions ループ制御用 Lambda 関数の準備

Step Functions ワークフローでループ処理をするために必要な Lambda 関数を作成します。

In [ ]:
lambda_sfn_loop_function_name  = project_name + 'lambda-sfn-loop-' + user_name
lambda_sfn_loop_policy_name = lambda_sfn_loop_function_name + '-policy'
lambda_sfn_loop_role_name = lambda_sfn_loop_function_name + '-role'
lambda_sfn_loop_json_name = 'lambda-sfn-loop-policy.json'

assume_role_policy = {
  "Version": "2012-10-17",
  "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"lambda.amazonaws.com"},"Action": "sts:AssumeRole"}]
}

lambda_sfn_loop_role_arn = utility.create_policy_role(
                    lambda_sfn_loop_policy_name, lambda_sfn_loop_json_name,
                    lambda_sfn_loop_role_name, assume_role_policy,
                    role_name_list, policy_arn_list)
sleep(10) # wait until IAM is created

以下のセルでは、Lambda 関数で使用するライブラリとソースコードを zip に固めています。このサンプルでは特にライブラリをインストールする必要はありませんが、ライブラリをインストールする際は、以下の処理を実行した環境と同じ Python のバージョンのランタイムを指定してください。2022年8月現在、conda_python3 カーネルの Python バージョンは 3.8 なので、Lambda 関数の Python バージョンも 3.8 を指定します。

In [ ]:
def prepare_lambda_resource(function_name, code_path):
    !rm -rf $function_name
    !rm {function_name}.zip
    !mkdir $function_name
#     !pip install pyyaml -t $function_name  # ライブラリのインストール例
    !cp {code_path}/index.py $function_name
    !cd $function_name && zip -r ../{function_name}.zip .
prepare_lambda_resource(lambda_sfn_loop_function_name, 'code/lambda')

作成した zip ファイルを使って Lambda 関数を作成します。

In [ ]:
lambda_sfn_loop_function_arn = utility.create_lambda_function(lambda_sfn_loop_function_name,
                                                   lambda_sfn_loop_function_name,
                                                   lambda_sfn_loop_role_arn,
                                                   'index',
                                                   lambda_function_list,
                                                   py_version='python3.8')

## AWS Step Functions の準備

あらかじめ用意してある JSON 形式の定義ファイルを使って、冒頭に示した Step Functions Workflow を作成します。

### IAM Role と Policy の作成

Step Functions の Workflow にセットする IAM Role を作成します。

In [ ]:
import json

step_functions_policy_name = project_name + '-sfn-' + user_name + '-policy'
step_functions_role_name = project_name + '-sfn-' + user_name + '-role'
step_functions_policy_json_name = 'stepfunctions-policy.json'

assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"states.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }

workflow_execution_role = utility.create_policy_role(
                    step_functions_policy_name, step_functions_policy_json_name,
                    step_functions_role_name, assume_role_policy,
                    role_name_list, policy_arn_list)
workflow_execution_role

### Step Functions ワークフローの作成

学習ジョブが正常終了しない場合、DescribeTrainingJob API を実行した結果が Lambda 関数に渡されるので、SNS への通知などエラーの種類に応じた処理をすることができます。ワークフロー実行時の入力パラメタを追加、変更したい場合は asl.json を変更してください。

In [ ]:
from botocore.exceptions import ClientError

asl_file = 'asl.json'
workflow_name = project_name + '-' + user_name
try:
    response = sfn_client.create_state_machine(
        name=workflow_name,
        definition=open(asl_file).read(),
        roleArn=workflow_execution_role,
        type='STANDARD'
    )
    workflow_arn = response['stateMachineArn']
    print('Workflow created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'StateMachineAlreadyExists':
        workflow_arn = utility.get_sfn_workflow_arn(workflow_name)
        response = sfn_client.update_state_machine(
            stateMachineArn=workflow_arn,
            definition=open(asl_file).read(),
            roleArn=workflow_execution_role
        )
        print('Workflow updated.')
    else:
        print(e)

workflow_arn

## Step Functions ワークフローの実行

実行時パラメタを指定して、ワークフローを実行します。

In [ ]:
from stepfunctions.workflow import Workflow
workflow = Workflow.attach(workflow_arn)

sfn_timestamp = datetime.now(JST).strftime('%Y%m%d-%H%M%S')
job_name = project_name + '-' + user_name + '-' + sfn_timestamp

checkpoint_s3_path = f's3://{bucket_name}/{prefix}/checkpoint'

execution = workflow.execute(
    inputs={
        # Step Functions Workflow Settings
        "counter": 0,  # カウンタ初期化
        "LambdaFunctionARN": f"{lambda_sfn_loop_function_arn}:$LATEST",
        # SageMaker Settings
        "EnableManagedSpotTraining": "true",
        "TrainingJobName": job_name,
        "TrainingImage": pytorch_training_image_uri,
        "S3OutputPath": f"s3://{bucket_name}/{prefix}",
        "RoleArn": role,
        "TrainingParameters": {
            "sagemaker_program": "train.py",
            "sagemaker_submit_directory": train_code,
            "epochs": "5"
        },
        "TrainingDataS3Path": inputs,
        "CheckPointS3Path": checkpoint_s3_path,
        "SpotMaxRuntimeInSeconds": 10,
        "SpotMaxWaitTimeInSeconds": 10,
        "OndemandMaxRuntimeInSeconds": 60*60*24,  # 24 hours
#         "StoppingCondition":{
#           "MaxRuntimeInSeconds": 60,
#           "MaxWaitTimeInSeconds": 60
#         },
        "InstanceList": [  # 使用したいインスタンス情報
            [
              "ml.m5.xlarge",
              "spot"
            ],
            [
              "ml.p3.2xlarge",
              "spot"
            ],
            [
              "ml.c5.xlarge",
              "ondemand"
            ]
        ]
    }
)
from IPython.display import display, Markdown
display(Markdown(f"<a href=\"https://{region}.console.aws.amazon.com/states/home?region={region}#/executions/details/{execution.execution_arn}\" target=\"_blank\">Step Functions のコンソール</a>"))

### Step Functions Workflow の動作確認

上記セルを実行した際に表示されたリンクから AWS コンソールに移動して今実行した Workflow を確認してみましょう。

## リソースの削除

今回作成したリソースは基本的に利用時のみに料金が発生するものですが、意図しない課金を防ぐために、不要になったらこのノートブックで作成したリソースを削除しましょう。

### Step Functions Workflow の削除

In [ ]:
workflow_list = Workflow.list_workflows()
workflow_arn = [d['stateMachineArn'] for d in workflow_list  if d['name']==workflow_name][0]
sfn_workflow = Workflow.attach(workflow_arn)
try:
    sfn_workflow.delete()
    print('Delete:', workflow_name)
except Exception as e:
    print(e)

### Lambda 関数の削除

In [ ]:
lambda_function_list = list(set(lambda_function_list))
for f in lambda_function_list:
    lambda_client.delete_function(FunctionName=f)

### S3 バケットの削除

S3 バケットを削除したい場合は、以下のセルのコメントアウトを外してから実行してバケットを空にしてください。その後、S3 のコンソールからバケットの削除を実行してください。

In [ ]:
# def delete_all_keys_v2(bucket, prefix, dryrun=False):
#     contents_count = 0
#     marker = ''

#     while True:
#         if marker == '':
#             response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix)
#         else:
#             response = s3_client.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=marker)

#         if 'Contents' in response:
#             contents = response['Contents']
#             contents_count = contents_count + len(contents)
#             for content in contents:
#                 if not dryrun:
#                     print("Deleting: s3://" + bucket + "/" + content['Key'])
#                     s3_client.delete_object(Bucket=bucket, Key=content['Key'])
#                 else:
#                     print("DryRun: s3://" + bucket + "/" + content['Key'])

#         if 'NextContinuationToken' in response:
#             marker = response['NextContinuationToken']
#         else:
#             break

#     print(contents_count, 'file were deleted.')

# delete_all_keys_v2(bucket_name, '')

### IAM Role と Policy の削除

In [ ]:
role_name_list = list(set(role_name_list))
policy_arn_list = list(set(policy_arn_list))

utility.delete_role_policy(role_name_list, policy_arn_list)

# ノートブックインスタンスにアタッチしたポリシーの削除
sagemaker_policy_arn = utility.get_policy_arn(sagemaker_policy_name)
response = iam_client.detach_role_policy(
    RoleName=role.split('/')[2],
    PolicyArn=sagemaker_policy_arn
)
print('\nこちらの IAM Policy は手動で削除してください。', sagemaker_policy_arn)